In [1]:
# Import the usual libraries
%pylab
%matplotlib inline
import os

plt.style.use('MSUstyle')

# Import sarkas
from sarkas.processes import PostProcess
# Create the file path to the YAML input file
example_directory = os.path.join('..',
                                 os.path.join('sarkas',
                                              os.path.join('examples','Coulomb')))

input_file_name = os.path.join(example_directory, 'BIM_cgs.yaml')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Let's initialize the class
postproc = PostProcess(input_file_name)
postproc.setup(read_yaml=True)




* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
                                 Postprocessing                                 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *

Job ID:  bim_gamma_40
Job directory:  Simulations/bim_gamma_40
Equilibration dumps directory:  Simulations/bim_gamma_40/Simulation/Equilibration/dumps
Production dumps directory:  Simulations/bim_gamma_40/Simulation/Production/dumps
PostProcessing directory:  Simulations/bim_gamma_40/PostProcessing


In [51]:
def kspace_setup(box_lengths, angle_averaging, max_k_harmonics, max_aa_harmonics):
    """
    Calculate all allowed :math:`k` vectors.

    Parameters
    ----------
    max_k_harmonics : numpy.ndarray
        Number of harmonics in each direction.

    box_lengths : numpy.ndarray
        Length of each box's side.

    angle_averaging : str
        Flag for calculating all the possible `k` vector directions and magnitudes. Default = 'principal_axis'

    max_aa_harmonics : numpy.ndarray
        Maximum `k` harmonics in each direction for angle average.

    Returns
    -------
    k_arr : list
        List of all possible combination of :math:`(n_x, n_y, n_z)` with their corresponding magnitudes and indexes.

    k_counts : numpy.ndarray
        Number of occurrences of each triplet :math:`(n_x, n_y, n_z)` magnitude.

    k_unique : numpy.ndarray
        Magnitude of the unique allowed triplet :math:`(n_x, n_y, n_z)`.
    """
    if angle_averaging == 'full':
        # The first value of k_arr = [0, 0, 0]
        first_non_zero = 1
        # Obtain all possible permutations of the wave number arrays
        k_arr = [np.array([i / box_lengths[0], j / box_lengths[1], k / box_lengths[2]])
                 for i in range(max_k_harmonics[0] + 1)
                 for j in range(max_k_harmonics[1] + 1)
                 for k in range(max_k_harmonics[2] + 1)]
        harmonics = [np.array([i, j, k], dtype=np.int)
                     for i in range(max_k_harmonics[0] + 1)
                     for j in range(max_k_harmonics[1] + 1)
                     for k in range(max_k_harmonics[2] + 1)]
    elif angle_averaging == 'principal_axis':
        # The first value of k_arr = [1, 0, 0]
        first_non_zero = 0
        # Calculate the k vectors along the principal axis only
        k_arr = [np.array([i / box_lengths[0], 0, 0]) for i in range(1, max_k_harmonics[0] + 1)]
        harmonics = [np.array([i, 0, 0], dtype=np.int) for i in range(1, max_k_harmonics[0] + 1)]
        k_arr = np.append(k_arr,
                          [np.array([0, i / box_lengths[1], 0]) for i in range(1, max_k_harmonics[1] + 1)],
                          axis=0)
        harmonics = np.append(harmonics,
                              [np.array([0, i, 0], dtype=np.int) for i in range(1, max_k_harmonics[1] + 1)],
                              axis=0)

        k_arr = np.append(k_arr,
                          [np.array([0, 0, i / box_lengths[2]]) for i in range(1, max_k_harmonics[2] + 1)],
                          axis=0)
        harmonics = np.append(harmonics,
                              [np.array([0, 0, i], dtype=np.int) for i in range(1, max_k_harmonics[2] + 1)],
                              axis=0)

    elif angle_averaging == 'custom':
        # The first value of k_arr = [0, 0, 0]
        first_non_zero = 1
        # Obtain all possible permutations of the wave number arrays up to max_aa_harmonics included
        k_arr = [np.array([i / box_lengths[0],
                           j / box_lengths[1],
                           k / box_lengths[2]]) for i in range(max_aa_harmonics[0] + 1)
                 for j in range(max_aa_harmonics[1] + 1)
                 for k in range(max_aa_harmonics[2] + 1)]
        # Append the rest of k values calculated from principal axis
        k_arr = np.append(
            k_arr,
            [np.array([i / box_lengths[0], 0, 0]) for i in range(max_aa_harmonics[0] + 1, max_k_harmonics[0] + 1)],
            axis=0)
        k_arr = np.append(
            k_arr,
            [np.array([0, i / box_lengths[1], 0]) for i in range(max_aa_harmonics[1] + 1, max_k_harmonics[1] + 1)],
            axis=0)
        k_arr = np.append(
            k_arr,
            [np.array([0, 0, i / box_lengths[2]]) for i in range(max_aa_harmonics[2] + 1, max_k_harmonics[2] + 1)],
            axis=0)
    
    k_arr2 = np.copy(k_arr)
    # Compute wave number magnitude - don't use |k| (skipping first entry in k_arr)
    k_mag = np.sqrt(np.sum(np.array(k_arr) ** 2, axis=1)[..., None])
    harm_mag = np.sqrt(np.sum(np.array(harmonics) ** 2, axis=1)[..., None])
    
    for i, k in enumerate(k_mag[: -1]):

        if abs(k - k_mag[i + 1]) < 2e-5:
            k_mag[i + 1] = k
        
    # Add magnitude to wave number array
    k_arr = np.concatenate((k_arr, k_mag), 1)
    # Add magnitude to wave number array
    harmonics = np.concatenate((harmonics, harm_mag), 1)

    # Sort from lowest to highest magnitude
    ind = np.argsort(k_arr[:, -1])
    k_arr = k_arr[ind]
    harmonics = harmonics[ind]

    # Count how many times a |k| value appears
    k_unique, k_counts = np.unique(k_arr[first_non_zero:, -1], return_counts=True)

    # Generate a 1D array containing index to be used in S array
    k_index = np.repeat(range(len(k_counts)), k_counts)[..., None]

    # Add index to k_array
    k_arr = np.concatenate((k_arr[int(first_non_zero):, :], k_index), 1)
    harmonics = np.concatenate((harmonics[int(first_non_zero):, :], k_index), 1)

    return k_arr, k_counts, k_unique, harmonics, k_arr2

IndentationError: expected an indented block (<ipython-input-51-84a7d82062a1>, line 91)

In [31]:
print(postproc.parameters.box_lengths[2])
k_list, k_counts, k_unique, harmonics, k_arr2 = kspace_setup(
    [3.9521036716556303e-10, 3.9521036716556303e-10, 3.9521036716556303e-10],
    'full', 
    [5,5,5],
    [5,5,5]
)

3.95210367165563e-09


In [50]:
# Compute wave number magnitude - don't use |k| (skipping first entry in k_arr)
k_mag2 = np.sqrt(np.sum(np.array(k_arr2) ** 2, axis=1)[..., None])
for i, k in enumerate(k_mag2[: -1]):
    print(i, k)
    if abs(k - k_mag2[i + 1]) < 2e-5:
        k_mag2[i + 1] = k
# # Add magnitude to wave number array
# k_arr3 = np.concatenate((k_arr2, k_mag2), 1)

# # Sort from lowest to highest magnitude
# ind = np.argsort(k_arr3[:, -1])
# k_arr3 = k_arr3[ind]

# # Count how many times a |k| value appears
# k_unique2, k_counts2 = np.unique(k_mag2[1:], return_counts=True)

0 [0.]
1 [2.530298e+09]
2 [5.06059599e+09]
3 [7.59089399e+09]
4 [1.0121192e+10]
5 [1.265149e+10]
6 [2.530298e+09]
7 [3.57838174e+09]
8 [5.65791832e+09]
9 [8.00150483e+09]
10 [1.04326859e+10]
11 [1.29020389e+10]
12 [5.06059599e+09]
13 [5.65791832e+09]
14 [7.15676349e+09]
15 [9.12311917e+09]
16 [1.13158366e+10]
17 [1.36260717e+10]
18 [7.59089399e+09]
19 [8.00150483e+09]
20 [9.12311917e+09]
21 [1.07351452e+10]
22 [1.265149e+10]
23 [1.47540459e+10]
24 [1.0121192e+10]
25 [1.04326859e+10]
26 [1.13158366e+10]
27 [1.265149e+10]
28 [1.4313527e+10]
29 [1.62018124e+10]
30 [1.265149e+10]
31 [1.29020389e+10]
32 [1.36260717e+10]
33 [1.47540459e+10]
34 [1.62018124e+10]
35 [1.78919087e+10]
36 [2.530298e+09]
37 [3.57838174e+09]
38 [5.65791832e+09]
39 [8.00150483e+09]
40 [1.04326859e+10]
41 [1.29020389e+10]
42 [3.57838174e+09]
43 [4.38260469e+09]
44 [6.19793899e+09]
45 [8.39204906e+09]
46 [1.07351452e+10]
47 [1.31478141e+10]
48 [5.65791832e+09]
49 [6.19793899e+09]
50 [7.59089399e+09]
51 [9.46750819e+09]

In [36]:
a_ws = 1.1379734387588278e-11
k_unique

array([2.53029800e+09, 3.57838174e+09, 4.38260469e+09, 5.06059599e+09,
       5.65791832e+09, 6.19793899e+09, 7.15676349e+09, 7.59089399e+09,
       8.00150483e+09, 8.39204906e+09, 8.76520937e+09, 9.12311917e+09,
       9.46750819e+09, 1.01211920e+10, 1.04326859e+10, 1.04326859e+10,
       1.07351452e+10, 1.10293133e+10, 1.13158366e+10, 1.15952821e+10,
       1.18681496e+10, 1.23958780e+10, 1.26514900e+10, 1.29020389e+10,
       1.31478141e+10, 1.36260717e+10, 1.38590129e+10, 1.43135270e+10,
       1.45354554e+10, 1.47540459e+10, 1.49694448e+10, 1.51817880e+10,
       1.55978044e+10, 1.62018124e+10, 1.63982052e+10, 1.65922736e+10,
       1.69737550e+10, 1.75304187e+10, 1.78919087e+10, 1.80699420e+10,
       1.85938170e+10, 1.91033309e+10, 1.94355877e+10, 2.05562381e+10,
       2.19130234e+10])

In [47]:
print(k_list[155,-2])
print(k_list[156,-2])
print(k_list[157,-2])
print(k_list[158,-2])
print(k_list[159,-2])

14969444818.79249
15181787975.43146
15181787975.43146
15181787975.43146
15597804397.64262


In [44]:
k_list[156][-2]

15181787975.43146

In [16]:
harmonics[16]

array([1.        , 1.        , 2.        , 2.44948974, 5.        ])

In [17]:
print(harmonics[16][-2])
print(harmonics[17][-2])

2.449489742783178
2.449489742783178


In [11]:
k_list[17][-2]

619793898.7154729